## En caso de no tener las librerías necesarias instaladas

In [ ]:
!pip install -r requirements.txt

## Imports

In [ ]:
import os
os.environ["TRITON_PRINT_AUTOTUNING"] = "1"
import torch
import triton
import triton.language as tl
import time
import numpy as np
import cupy as cp
import itertools
from statistics import mean, median
import matplotlib.pyplot as plt

## Configuración autotuning

In [ ]:
#Comentar/Descomentar en funcion del tipo de autotuning deseado (grid search (1ª opcion) o pre-configuraciones (2ª opcion))

#Grid search
'''
# Rango de valores a explorar
block_size_ms = [32, 64, 128,256]
block_size_ns = [32, 64, 128,256]
block_size_ks = [16, 32, 64, 128, 256]
num_warps_list = [4]
num_stages_list = [2]
group_sizes = [8]

# Generar todas las combinaciones posibles como un grid search

autotune_configs = [
    triton.Config(
        {
            "BLOCK_SIZE_M": bm,
            "BLOCK_SIZE_N": bn,
            "BLOCK_SIZE_K": bk,
            "GROUP_SIZE": g
        },
        num_warps=w,
        num_stages=s
    )
    for bm, bn, bk, g, w, s in itertools.product(
        block_size_ms, block_size_ns, block_size_ks,
        group_sizes, num_warps_list, num_stages_list
    )
]
'''

#Pre-configs
autotune_configs = [
    triton.Config({"BLOCK_SIZE_M": 128, "BLOCK_SIZE_N": 256, "BLOCK_SIZE_K": 64, "GROUP_SIZE": 8}, num_stages=3, num_warps=8),
    triton.Config({"BLOCK_SIZE_M": 64, "BLOCK_SIZE_N": 256, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 128, "BLOCK_SIZE_N": 128, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 128, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 64, "BLOCK_SIZE_N": 128, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 128, "BLOCK_SIZE_N": 32, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 64, "BLOCK_SIZE_N": 32, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=5, num_warps=2),
    triton.Config({"BLOCK_SIZE_M": 32, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=5, num_warps=2),
    triton.Config({"BLOCK_SIZE_M": 32, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=5, num_warps=2),
    triton.Config({"BLOCK_SIZE_M": 32, "BLOCK_SIZE_N": 32, "BLOCK_SIZE_K": 16, "GROUP_SIZE": 4}, num_stages=4, num_warps=2),
    triton.Config({"BLOCK_SIZE_M": 64, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 16, "GROUP_SIZE": 8}, num_stages=3, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 256, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=3, num_warps=8),
]

## Implementaciones del kernel de GEMM

## FP16

In [ ]:
DEVICE = torch.device(f'cuda:{0}') #Seleccionamos la GPU sobre la que se ejecuta el codigo
torch.manual_seed(0) #Para obtener siempre los mismos resultados

#Kernel Triton
@triton.autotune(configs=autotune_configs, key=["M", "N", "K"])
@triton.jit
def matmul_kernel(
    A_ptr,
    B_ptr,
    C_ptr,
    M,
    N,
    K,
    stride_a_M,
    stride_a_K,
    stride_b_K,
    stride_b_N,
    stride_c_M,
    stride_c_N,
    BLOCK_SIZE_M: tl.constexpr,
    BLOCK_SIZE_N: tl.constexpr,
    BLOCK_SIZE_K: tl.constexpr,
    GROUP_SIZE: tl.constexpr,
    num_stages: tl.constexpr,
    num_warps: tl.constexpr,
):

    PID = tl.program_id(0)
    num_PID_along_M = tl.cdiv(M, BLOCK_SIZE_M)
    num_PID_along_N = tl.cdiv(N, BLOCK_SIZE_N)
    num_PID_in_group = GROUP_SIZE * num_PID_along_N
    group_id = PID // num_PID_in_group
    first_PID_in_group_along_M = group_id * GROUP_SIZE
    group_size_adj = min(num_PID_along_M - first_PID_in_group_along_M, GROUP_SIZE)
    PID_M = first_PID_in_group_along_M + ((PID % num_PID_in_group) % group_size_adj)
    PID_N = ((PID % num_PID_in_group) // group_size_adj)

    offsets_M = PID_M * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offsets_N = PID_N * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    offsets_K = tl.arange(0, BLOCK_SIZE_K)

    a_offsets = offsets_M[:,None] * stride_a_M + offsets_K[None,:] * stride_a_K #[[m1n1],[m1n2],[m1,n3]] Resultados "visuales" de estas operaciones
                                                                                #[[m2n1],[m2n2],[m2,n3]]
                                                                                #[[m3n1],[m3n2],[m3,n3]]
    b_offsets = offsets_K[:,None] * stride_b_K + offsets_N[None,:] * stride_b_N #[[k1n1],[k1n2],[k1,n3]]

    accumulator = tl.zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=tl.float32)

    for k in range(0, tl.cdiv(K, BLOCK_SIZE_K)):
        mask = offsets_K < K - k * BLOCK_SIZE_K
        a_c = tl.load(A_ptr + a_offsets, mask=mask[None,:],other=0.0)
        b_c = tl.load(B_ptr + b_offsets, mask=mask[:,None],other=0.0)

        accumulator = tl.dot(a_c,b_c,acc=accumulator) #tl.dot hace un matmul. Es como hacer a@b

        a_offsets +=  BLOCK_SIZE_K * stride_a_K
        b_offsets +=  BLOCK_SIZE_K * stride_b_K

    c_offsets = offsets_M[:,None] * stride_c_M + offsets_N[None,:] * stride_c_N
    c_mask = (offsets_M[:,None] < M) & (offsets_N[None,:] < N)
    tl.store(C_ptr + c_offsets, accumulator.to(tl.float16), mask=c_mask)


def matmul_tri(a,b,c,M,N,K,grid): #Funcion wrapper del kernel de triton
    matmul_kernel[grid](a,b,c,
                        M,N,K,
                        a.stride(0),a.stride(1),
                        b.stride(0),b.stride(1),
                        c.stride(0),c.stride(1),)
    return c


def matmul_cu(a,b): #Funcion wrapper CuPy
    c_cp = cp.matmul(a,b)
    return c_cp

def matmul_cb(a,b): #Funcion wrapper del kernel de torch
    c_cb = torch.matmul(a,b)
    return c_cb

def ejecucion_capa(M,N,K,num_repeticiones=100):
    tiempos_triton = []
    tiempos_cupy = []
    tiempos_cublas = []
    #Tensores triton
    a_tri = torch.rand(M, K, device=DEVICE, dtype=torch.float16)
    b_tri = torch.rand(K, N, device=DEVICE, dtype=torch.float16)
    c_tri = torch.rand(M, N, device=DEVICE, dtype=torch.float16)
    grid = lambda meta: (triton.cdiv(M, meta['BLOCK_SIZE_M']) * triton.cdiv(N, meta['BLOCK_SIZE_N']),)

    #Matrices CuPy
    a_cp = cp.asarray(a_tri.cpu().numpy()).astype(cp.float16)
    b_cp = cp.asarray(b_tri.cpu().numpy()).astype(cp.float16)

    #Tensores cuBLAS (torch)
    a_cb = a_tri.to(torch.float16)
    b_cb = b_tri.to(torch.float16)

    #Primera ejecución para la compilación
    #Triton
    torch.cuda.synchronize()
    start_tri = time.time()
    c_tri = matmul_tri(a_tri,b_tri,c_tri,M,N,K,grid)
    torch.cuda.synchronize()
    end_tri = time.time()
    print(f"Tiempo compilación Triton: {end_tri-start_tri:.6f} s")
    #CuPy
    cp.cuda.Device(0).synchronize()
    start_cp = time.time()
    c_cp = matmul_cu(a_cp,b_cp)
    cp.cuda.Device(0).synchronize()
    end_cp = time.time()
    print(f"Tiempo compilación CuPy: {end_cp-start_cp:.6f} s")
    #cuBLAS
    torch.cuda.synchronize()
    start_cublas = time.time()
    c_cb = matmul_cb(a_cb,b_cb)
    torch.cuda.synchronize()
    end_cublas = time.time()
    print(f"Tiempo compilacion cuBLAS: {end_cublas-start_cublas:.6f} s")


    #Comprobación de que la operación realizada es la misma.
    c_cp = torch.tensor(cp.asnumpy(c_cp), device=DEVICE, dtype=torch.float16)
    #Comprobacion con CuPy
    torch.testing.assert_close(c_tri, c_cp, atol=1e-3, rtol=1e-3)
    #Comprobacion con cuBLAS
    torch.testing.assert_close(c_tri, c_cb, atol=1e-3, rtol=1e-3)

    # Repeticiones para medición
    for _ in range(num_repeticiones):
        # Medir Triton
        torch.cuda.synchronize()
        start_tri = time.time()
        c_tri = matmul_tri(a_tri, b_tri, c_tri, M, N, K, grid)
        torch.cuda.synchronize()
        end_tri = time.time()
        tiempos_triton.append(end_tri - start_tri)

        # Medir CuPy
        cp.cuda.Device(0).synchronize()
        start_cp = time.time()
        c_cp = matmul_cu(a_cp, b_cp)
        cp.cuda.Device(0).synchronize()
        end_cp = time.time()
        tiempos_cupy.append(end_cp - start_cp)

        # Medir cuBLAS
        torch.cuda.synchronize()
        start_cublas = time.time()
        c_cb = c_cublas = matmul_cb(a_cb,b_cb)
        torch.cuda.synchronize()
        end_cublas = time.time()
        tiempos_cublas.append(end_cublas - start_cublas)

    # Calcular estadísticas
    stats_triton = {
        'min': min(tiempos_triton),
        'max': max(tiempos_triton),
        'media': mean(tiempos_triton),
        'mediana': median(tiempos_triton)
    }

    stats_cupy = {
        'min': min(tiempos_cupy),
        'max': max(tiempos_cupy),
        'media': mean(tiempos_cupy),
        'mediana': median(tiempos_cupy)
    }

    stats_cublas = {
        'min': min(tiempos_cublas),
        'max': max(tiempos_cublas),
        'media': mean(tiempos_cublas),
        'mediana': median(tiempos_cublas)
    }

    return stats_triton, stats_cupy, stats_cublas

## INT 8

In [ ]:
DEVICE = torch.device(f'cuda:{0}') #Seleccionamos la GPU sobre la que se ejecuta el codigo
torch.manual_seed(0) #Para obtener siempre los mismos resultados

#Kernel Triton
@triton.autotune(configs=autotune_configs, key=["M", "N", "K"])
@triton.jit
def matmul_kernel_int8(
    A_ptr,
    B_ptr,
    C_ptr,
    M,
    N,
    K,
    stride_a_M,
    stride_a_K,
    stride_b_K,
    stride_b_N,
    stride_c_M,
    stride_c_N,
    BLOCK_SIZE_M: tl.constexpr,
    BLOCK_SIZE_N: tl.constexpr,
    BLOCK_SIZE_K: tl.constexpr,
    GROUP_SIZE: tl.constexpr,
    num_stages: tl.constexpr,
    num_warps: tl.constexpr,
):

    PID = tl.program_id(0)
    num_PID_along_M = tl.cdiv(M, BLOCK_SIZE_M)
    num_PID_along_N = tl.cdiv(N, BLOCK_SIZE_N)
    num_PID_in_group = GROUP_SIZE * num_PID_along_N
    group_id = PID // num_PID_in_group
    first_PID_in_group_along_M = group_id * GROUP_SIZE
    group_size_adj = min(num_PID_along_M - first_PID_in_group_along_M, GROUP_SIZE)
    PID_M = first_PID_in_group_along_M + ((PID % num_PID_in_group) % group_size_adj)
    PID_N = ((PID % num_PID_in_group) // group_size_adj)

    offsets_M = PID_M * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offsets_N = PID_N * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    offsets_K = tl.arange(0, BLOCK_SIZE_K)

    a_offsets = offsets_M[:,None] * stride_a_M + offsets_K[None,:] * stride_a_K #[[m1n1],[m1n2],[m1,n3]] Resultados "visuales" de estas operaciones
                                                                                #[[m2n1],[m2n2],[m2,n3]]
                                                                                #[[m3n1],[m3n2],[m3,n3]]
    b_offsets = offsets_K[:,None] * stride_b_K + offsets_N[None,:] * stride_b_N #[[k1n1],[k1n2],[k1,n3]]

    accumulator = tl.zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=tl.int32)

    for k in range(0, tl.cdiv(K, BLOCK_SIZE_K)):
        mask = offsets_K < K - k * BLOCK_SIZE_K
        a_c = tl.load(A_ptr + a_offsets, mask=mask[None,:],other=0)
        b_c = tl.load(B_ptr + b_offsets, mask=mask[:,None],other=0)

        accumulator = tl.dot(a_c,b_c,acc=accumulator) #tl.dot hace un matmul. Es como hacer a@b

        a_offsets +=  BLOCK_SIZE_K * stride_a_K
        b_offsets +=  BLOCK_SIZE_K * stride_b_K

    c_offsets = offsets_M[:,None] * stride_c_M + offsets_N[None,:] * stride_c_N
    c_mask = (offsets_M[:,None] < M) & (offsets_N[None,:] < N)
    tl.store(C_ptr + c_offsets, accumulator.to(tl.int8), mask=c_mask)


def matmul_tri_int8(a,b,c,M,N,K,grid):
    matmul_kernel_int8[grid](a,b,c,
                        M,N,K,
                        a.stride(0),a.stride(1),
                        b.stride(0),b.stride(1),
                        c.stride(0),c.stride(1),)
    return c

def matmul_cu(a,b): #Funcion wrapper CuPy
    c_cp = cp.matmul(a,b)
    return c_cp

def ejecucion_capa_int8(M,N,K,num_repeticiones=100):
    tiempos_triton = []
    tiempos_cupy = []
    tiempos_cublas = []
    #Tensores triton
    a_tri = torch.randint(-128, 128, (M, K), device=DEVICE, dtype=torch.int8)
    b_tri = torch.randint(-128, 128, (K, N), device=DEVICE, dtype=torch.int8)
    c_tri = torch.zeros(M, N, device=DEVICE, dtype=torch.int8)
    grid = lambda meta: (triton.cdiv(M, meta['BLOCK_SIZE_M']) * triton.cdiv(N, meta['BLOCK_SIZE_N']),)

    #Matrices CuPy
    a_cp = cp.asarray(a_tri.cpu().numpy()).astype(cp.int8)
    b_cp = cp.asarray(b_tri.cpu().numpy()).astype(cp.int8)

    #Primera ejecución para la compilación
    #Triton
    torch.cuda.synchronize()
    start_tri = time.time()
    c_tri = matmul_tri_int8(a_tri,b_tri,c_tri,M,N,K,grid)
    torch.cuda.synchronize()
    end_tri = time.time()
    print(f"Tiempo compilación Triton: {end_tri-start_tri:.6f} s")
    #CuPy
    cp.cuda.Device(0).synchronize()
    start_cp = time.time()
    c_cp = matmul_cu(a_cp,b_cp)
    cp.cuda.Device(0).synchronize()
    end_cp = time.time()
    print(f"Tiempo compilación CuPy: {end_cp-start_cp:.6f} s")


    #Comprobación de que la operación realizada es la misma.
    c_cp = torch.tensor(cp.asnumpy(c_cp), device=DEVICE, dtype=torch.int8)
    torch.testing.assert_close(c_tri, c_cp, atol=1e-2, rtol=0)
    print("El resultado de la operacion que realizan es correcto")

    # Repeticiones para medición
    for _ in range(num_repeticiones):
        # Medir Triton
        torch.cuda.synchronize()
        start_tri = time.time()
        c_tri = matmul_tri_int8(a_tri, b_tri, c_tri, M, N, K, grid)
        torch.cuda.synchronize()
        end_tri = time.time()
        tiempos_triton.append(end_tri - start_tri)

        # Medir CuPy
        cp.cuda.Device(0).synchronize()
        start_cp = time.time()
        c_cp = matmul_cu(a_cp, b_cp)
        cp.cuda.Device(0).synchronize()
        end_cp = time.time()
        tiempos_cupy.append(end_cp - start_cp)


    # Calcular estadísticas
    stats_triton = {
        'min': min(tiempos_triton),
        'max': max(tiempos_triton),
        'media': mean(tiempos_triton),
        'mediana': median(tiempos_triton)
    }

    stats_cupy = {
        'min': min(tiempos_cupy),
        'max': max(tiempos_cupy),
        'media': mean(tiempos_cupy),
        'mediana': median(tiempos_cupy)
    }

    return stats_triton, stats_cupy

## FP32

In [ ]:
DEVICE = torch.device(f'cuda:{0}')
torch.manual_seed(0) #Para obtener siempre los mismos resultados

autotune_configs = [
    triton.Config({"BLOCK_SIZE_M": 128, "BLOCK_SIZE_N": 256, "BLOCK_SIZE_K": 64, "GROUP_SIZE": 8}, num_stages=3, num_warps=8),
    triton.Config({"BLOCK_SIZE_M": 64, "BLOCK_SIZE_N": 256, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 128, "BLOCK_SIZE_N": 128, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 128, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 64, "BLOCK_SIZE_N": 128, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 128, "BLOCK_SIZE_N": 32, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=4, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 64, "BLOCK_SIZE_N": 32, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=5, num_warps=2),
    triton.Config({"BLOCK_SIZE_M": 32, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=5, num_warps=2),
    triton.Config({"BLOCK_SIZE_M": 32, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=5, num_warps=2),
    triton.Config({"BLOCK_SIZE_M": 32, "BLOCK_SIZE_N": 32, "BLOCK_SIZE_K": 16, "GROUP_SIZE": 4}, num_stages=4, num_warps=2),
    triton.Config({"BLOCK_SIZE_M": 64, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 16, "GROUP_SIZE": 8}, num_stages=3, num_warps=4),
    triton.Config({"BLOCK_SIZE_M": 256, "BLOCK_SIZE_N": 64, "BLOCK_SIZE_K": 32, "GROUP_SIZE": 8}, num_stages=3, num_warps=8),
]

@triton.autotune(configs=autotune_configs, key=["M", "N", "K"])
@triton.jit
def matmul_kernel_fp32(
    A_ptr,
    B_ptr,
    C_ptr,
    M,
    N,
    K,
    stride_a_M,
    stride_a_K,
    stride_b_K,
    stride_b_N,
    stride_c_M,
    stride_c_N,
    BLOCK_SIZE_M: tl.constexpr,
    BLOCK_SIZE_N: tl.constexpr,
    BLOCK_SIZE_K: tl.constexpr,
    GROUP_SIZE: tl.constexpr,
    num_stages: tl.constexpr,
    num_warps: tl.constexpr,
):

    PID = tl.program_id(0)
    num_PID_along_M = tl.cdiv(M, BLOCK_SIZE_M)
    num_PID_along_N = tl.cdiv(N, BLOCK_SIZE_N)
    num_PID_in_group = GROUP_SIZE * num_PID_along_N
    group_id = PID // num_PID_in_group
    first_PID_in_group_along_M = group_id * GROUP_SIZE
    group_size_adj = min(num_PID_along_M - first_PID_in_group_along_M, GROUP_SIZE)
    PID_M = first_PID_in_group_along_M + ((PID % num_PID_in_group) % group_size_adj)
    PID_N = ((PID % num_PID_in_group) // group_size_adj)

    offsets_M = PID_M * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offsets_N = PID_N * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    offsets_K = tl.arange(0, BLOCK_SIZE_K)

    a_offsets = offsets_M[:,None] * stride_a_M + offsets_K[None,:] * stride_a_K #[[m1n1],[m1n2],[m1,n3]]
                                                                                #[[m2n1],[m2n2],[m2,n3]]
                                                                                #[[m3n1],[m3n2],[m3,n3]]
    b_offsets = offsets_K[:,None] * stride_b_K + offsets_N[None,:] * stride_b_N #[[k1n1],[k1n2],[k1,n3]]

    accumulator = tl.zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=tl.float32)

    for k in range(0, tl.cdiv(K, BLOCK_SIZE_K)):
        mask = offsets_K < K - k * BLOCK_SIZE_K
        a_c = tl.load(A_ptr + a_offsets, mask=mask[None,:],other=0.0)
        b_c = tl.load(B_ptr + b_offsets, mask=mask[:,None],other=0.0)

        accumulator = tl.dot(a_c,b_c,acc=accumulator) #tl.dot hace un matmul. Es como hacer a@b

        a_offsets +=  BLOCK_SIZE_K * stride_a_K
        b_offsets +=  BLOCK_SIZE_K * stride_b_K

    c_offsets = offsets_M[:,None] * stride_c_M + offsets_N[None,:] * stride_c_N
    c_mask = (offsets_M[:,None] < M) & (offsets_N[None,:] < N)
    tl.store(C_ptr + c_offsets, accumulator, mask=c_mask)


def matmul_tri_fp32(a,b,c,M,N,K,grid):
    matmul_kernel_fp32[grid](a,b,c,
                        M,N,K,
                        a.stride(0),a.stride(1),
                        b.stride(0),b.stride(1),
                        c.stride(0),c.stride(1),)
    return c


def matmul_cu(a,b):
    c_cp = cp.matmul(a,b)
    return c_cp

def matmul_cb(a,b):
    c_cb = torch.matmul(a,b)
    return c_cb

def ejecucion_capa_fp32(M,N,K,num_repeticiones=100):
    tiempos_triton = []
    tiempos_cupy = []
    tiempos_cublas = []
    #Tensores triton
    a_tri = torch.rand(M, K, device=DEVICE, dtype=torch.float32)
    b_tri = torch.rand(K, N, device=DEVICE, dtype=torch.float32)
    c_tri = torch.rand(M, N, device=DEVICE, dtype=torch.float32)
    grid = lambda meta: (triton.cdiv(M, meta['BLOCK_SIZE_M']) * triton.cdiv(N, meta['BLOCK_SIZE_N']),)

    #Matrices CuPy
    a_cp = cp.asarray(a_tri.cpu().numpy()).astype(cp.float32)
    b_cp = cp.asarray(b_tri.cpu().numpy()).astype(cp.float32)

    #Tensores cuBLAS (torch)
    a_cb = a_tri.to(torch.float32)
    b_cb = b_tri.to(torch.float32)

    #Primera ejecución para la compilación
    #Triton
    torch.cuda.synchronize()
    start_tri = time.time()
    c_tri = matmul_tri_fp32(a_tri,b_tri,c_tri,M,N,K,grid)
    torch.cuda.synchronize()
    end_tri = time.time()
    print(f"Tiempo compilación Triton: {end_tri-start_tri:.6f} s")
    #CuPy
    cp.cuda.Device(0).synchronize()
    start_cp = time.time()
    c_cp = matmul_cu(a_cp,b_cp)
    cp.cuda.Device(0).synchronize()
    end_cp = time.time()
    print(f"Tiempo compilación CuPy: {end_cp-start_cp:.6f} s")
    #cuBLAS
    torch.cuda.synchronize()
    start_cublas = time.time()
    c_cb = matmul_cb(a_cb,b_cb)
    torch.cuda.synchronize()
    end_cublas = time.time()
    print(f"Tiempo compilacion cuBLAS: {end_cublas-start_cublas:.6f} s")


    #Comprobación de que la operación realizada es la misma.
    c_cp = torch.tensor(cp.asnumpy(c_cp), device=DEVICE, dtype=torch.float32)
    #Comprobacion con CuPy
    torch.cuda.empty_cache()
    torch.testing.assert_close(c_tri, c_cp, atol=1e-3, rtol=1e-3)
    #Comprobacion con cuBLAS
    torch.cuda.empty_cache()
    torch.testing.assert_close(c_tri, c_cb, atol=1e-3, rtol=1e-3)

    # Repeticiones para medición
    for _ in range(num_repeticiones):
        # Medir Triton
        torch.cuda.synchronize()
        start_tri = time.time()
        c_tri = matmul_tri_fp32(a_tri, b_tri, c_tri, M, N, K, grid)
        torch.cuda.synchronize()
        end_tri = time.time()
        tiempos_triton.append(end_tri - start_tri)

        # Medir CuPy
        cp.cuda.Device(0).synchronize()
        start_cp = time.time()
        c_cp = matmul_cu(a_cp, b_cp)
        cp.cuda.Device(0).synchronize()
        end_cp = time.time()
        tiempos_cupy.append(end_cp - start_cp)

        # Medir cuBLAS
        torch.cuda.synchronize()
        start_cublas = time.time()
        c_cb = c_cublas = matmul_cb(a_cb,b_cb)
        torch.cuda.synchronize()
        end_cublas = time.time()
        tiempos_cublas.append(end_cublas - start_cublas)

    # Calcular estadísticas
    stats_triton = {
        'min': min(tiempos_triton),
        'max': max(tiempos_triton),
        'media': mean(tiempos_triton),
        'mediana': median(tiempos_triton)
    }

    stats_cupy = {
        'min': min(tiempos_cupy),
        'max': max(tiempos_cupy),
        'media': mean(tiempos_cupy),
        'mediana': median(tiempos_cupy)
    }

    stats_cublas = {
        'min': min(tiempos_cublas),
        'max': max(tiempos_cublas),
        'media': mean(tiempos_cublas),
        'mediana': median(tiempos_cublas)
    }

    return stats_triton, stats_cupy, stats_cublas

## Creamos los diccionaros correspondientes a los modelos (VGG16 y ResNET50 v 1.5)

In [ ]:
VGG16_capas = {
    1:   [["01"], 50176, 64, 27],
    2:   [["03"], 50176, 64, 576],
    3:   [["06"], 12544, 128, 576],
    4:   [["08"], 12544, 128, 1152],
    5:   [["11"], 3136, 256, 1152],
    6:   [["13", "15"], 3136, 256, 2304],
    7:   [["18"], 784, 256, 2304],
    8:   [["20", "22"], 784, 512, 4608],
    9:   [["25", "27", "29"], 196, 512, 4608]
}

ResNET50_capas = {
    1:   [["001"], 12544, 64, 147],
    2:   [["006"], 3136, 64, 64],
    3:   [["009", "021", "031"], 3136, 64, 576],
    4:   [["012", "014", "024", "034"], 3136, 256, 64],
    5:   [["018", "028"], 3136, 64, 256],
    6:   [["038"], 3136, 128, 256],
    7:   [["041", "053", "063", "073"], 784, 128, 1152],
    8:   [["044", "056", "066", "076"], 784, 512, 128],
    9:   [["046"], 784, 512, 256],
    10:  [["050", "060", "070"], 784, 128, 512],
    11:  [["080"], 784, 256, 512],
    12:  [["083", "095", "105", "115", "125", "135"], 196, 256, 2304],
    13:  [["086", "098", "108", "118", "128", "138"], 196, 1024, 256],
    14:  [["088"], 196, 1024, 512],
    15:  [["092", "102", "112", "122", "132"], 196, 256, 1024],
    16:  [["142"], 196, 512, 1024],
    17:  [["145", "157", "167"], 49, 512, 4608],
    18:  [["148", "160", "170"], 49, 2048, 512],
    19:  [["150"], 49, 2048, 1024],
    20:  [["154", "164"], 49, 512, 2048]
}

## Creamos una función que facilite la ejecución de los modelos con distintos valores de parámetros

In [ ]:
def ejecutar_modelo(modelo,tipo_dato="fp16",tamano_batch=1,num_repeticiones=100):
  if tipo_dato=="fp16":
    funcion = ejecucion_capa
  elif tipo_dato=="int8":
    funcion = ejecucion_capa_int8
  else:
    funcion = ejecucion_capa_fp32
  resultados = {}
  for layer_id, (num_capas, m, n, k) in modelo.items():
      num_capas_str = ", ".join(num_capas)
      print("-" * 50)
      print(f"ID capa: {layer_id}")
      print(f"Tamaño de matrices: m = {tamano_batch * m}, n = {n}, k = {k}")
      print(f"Se emplea en las capas nº: {num_capas_str}")

      # Ejecutar mediciones
      if tipo_dato=="fp16" or tipo_dato=="fp32":
        stats_triton, stats_cupy, stats_cublas = funcion(tamano_batch * m, n, k, num_repeticiones)

        # Guardar resultados
        resultados[layer_id] = [
                [tamano_batch * m, n, k],
                [stats_triton['mediana'], stats_cupy['mediana'], stats_cublas['mediana']]
            ]
      elif tipo_dato=="int8":
        stats_triton, stats_cupy = funcion(tamano_batch * m, n, k, num_repeticiones)

        # Guardar resultados
        resultados[layer_id] = [
                [tamano_batch * m, n, k],
                [stats_triton['mediana'], stats_cupy['mediana']]
            ]

      # Imprimir resultados
      print("\nEstadísticas de tiempo (segundos):")
      print("Triton:")
      print(f"  Mínimo: {stats_triton['min']:.6f}")
      print(f"  Máximo: {stats_triton['max']:.6f}")
      print(f"  Media: {stats_triton['media']:.6f}")
      print(f"  Mediana: {stats_triton['mediana']:.6f}")

      print("\nCuPy:")
      print(f"  Mínimo: {stats_cupy['min']:.6f}")
      print(f"  Máximo: {stats_cupy['max']:.6f}")
      print(f"  Media: {stats_cupy['media']:.6f}")
      print(f"  Mediana: {stats_cupy['mediana']:.6f}")
      #Speedup
      speedup_cupy = stats_cupy['mediana'] / stats_triton['mediana']
      print(f"\nSpeedup (CuPy/Triton): {speedup_cupy:.2f}x")
      if tipo_dato=="fp16" or tipo_dato=="fp32":
        print("\ncuBLAS:")
        print(f"  Mínimo: {stats_cublas['min']:.6f}")
        print(f"  Máximo: {stats_cublas['max']:.6f}")
        print(f"  Media: {stats_cublas['media']:.6f}")
        print(f"  Mediana: {stats_cublas['mediana']:.6f}")
        #Speedup
        speedup_cublas = stats_cublas['mediana'] / stats_triton['mediana']
        print(f"Speedup (cuBLAS/Triton): {speedup_cublas:.2f}x")

      print("-" * 50)
  return resultados

## Ejecución principal (Iteramos sobre las distintas capas de los modelos llamando a las GEMM creadas anteriormente)

In [ ]:
resultados = ejecutar_modelo(VGG16_capas,tipo_dato="fp16",tamano_batch=1,num_repeticiones=100)

## Función para crear gráficos de resultado

In [ ]:
def crear_grafico(datos_capas, tipo, titulo, ruta_guardado):

    # Calcular TFLOPS/TOPS para cada capa e implementación
    capas = []
    valores_triton = []
    valores_cupy = []
    valores_cublas = [] if tipo == 'fp' else None

    for capa_id, datos in datos_capas.items():
        m, n, k = datos[0]
        if tipo == 'fp':
            tiempo_triton, tiempo_cupy, tiempo_cublas = datos[1]
        else:
            tiempo_triton, tiempo_cupy = datos[1]

        # operaciones TFLOPS/TIOPS
        operaciones = lambda tiempo: (2 * m * n * k) / (tiempo * 1e12)

        valores_triton.append(operaciones(tiempo_triton))
        valores_cupy.append(operaciones(tiempo_cupy))
        if tipo == 'fp':
            valores_cublas.append(operaciones(tiempo_cublas))
        capas.append(capa_id)

    # Crear el gráfico
    plt.figure(figsize=(15, 7))
    x = range(len(capas))

    if tipo == 'fp':
        width = 0.25
        # Crear barras para FP
        plt.bar([i - width for i in x], valores_triton, width, label='Triton', color='green')
        plt.bar(x, valores_cupy, width, label='CuPy', color='red')
        plt.bar([i + width for i in x], valores_cublas, width, label='cuBLAS', color='blue')
    else:
        width = 0.35
        # Crear barras para INT
        plt.bar([i - width/2 for i in x], valores_triton, width, label='Triton', color='green')
        plt.bar([i + width/2 for i in x], valores_cupy, width, label='CuPy', color='red')

    # Configurar el gráfico
    plt.xlabel('# of layer')
    plt.ylabel('TFLOPS' if tipo == 'fp' else 'TOPS')
    plt.title(titulo)
    plt.xticks(x, capas)
    plt.legend()

    # Añadir cuadrícula
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(ruta_guardado)
    plt.close()

    # Imprimir valores para verificación
    print("\nValores por capa:")
    for i, capa_id in enumerate(capas):
        print(f"\nCapa {capa_id}:")
        print(f"  Triton: {valores_triton[i]:.2f} {'TFLOPS' if tipo == 'fp' else 'TOPS'}")
        print(f"  CuPy: {valores_cupy[i]:.2f} {'TFLOPS' if tipo == 'fp' else 'TOPS'}")
        if tipo == 'fp':
            print(f"  cuBLAS: {valores_cublas[i]:.2f} TFLOPS")

## Guardar el gráfico en función del resultado

In [ ]:
crear_grafico(resultados,'fp','VGG16 - FP16 - Triton vs CuPy vs cuBLAS', 'VGG16-COLAB-FP16-B1.png')